In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import plotly.express as px
import polars as pl
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x :.2f}')
from cleaner import DataCleaner
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from tools import (
    logging,
    bins_generator,
    import_datasets,

)
import eda_movies as eda
clean = DataCleaner()

In [ ]:
# j'ai essayé de lancer ça ça a pris plus de 7 min j'ai give up

# others = test[~test["category"].isin([actors, directors])]

In [ ]:
actors = import_datasets("clean_datasets/actors.parquet", "parquet")

In [ ]:
directors = import_datasets("clean_datasets/directors.parquet", "parquet")

In [ ]:
link = "clean_datasets/person.parquet"
persons = import_datasets(link, "parquet")

In [ ]:
link = "clean_datasets/movies_cleaned.parquet"
movies = import_datasets(link, "parquet")

In [ ]:
actors_names = pd.merge(
    actors,
    persons,
    on = "nconst"
)

In [ ]:
actors_names.head()

In [ ]:
movies_actors = pd.merge(
    actors_names,
    movies,
    left_on = "tconst",
    right_on = "titre_id"
)

In [ ]:
movies_actors.drop(["tconst", "titleId"], inplace=True, axis=1)

In [ ]:
movies_actors.head()

In [ ]:
df = movies_actors.copy()

In [ ]:
column_order = [
    "titre_id",
    "titre_str",
    "titre_date_sortie",
    "titre_duree",
    "titre_genres",
    "rating_avg",
    "rating_votes",
    "original_language",
    "original_title",
    "popularity",
    "production_countries",
    "revenue",
    "spoken_languages",
    "status",
    "region",
    "cuts",
    "nconst",
    "category",
    "characters",
    "primaryName",
    "birthYear",
    "knownForTitles",
    "ordering"
]

df = df[column_order]

print(df.head().to_markdown())

In [ ]:
df["primaryName"].value_counts()

**JME SUIS ARRETÉ LA**


In [ ]:
df.to_parquet("clean_datasets/actors_movies.parquet")

In [ ]:
df = pd.read_parquet("clean_datasets/actors_movies.parquet")

In [ ]:
condi2 = (
    (df["original_title"].str.contains("Lord of the Ring"))
    # (df["nconst"].str.contains("nm0000367"))
)
df[condi2].head()

In [ ]:
grouped_df = df.groupby(
    ['cuts', 'primaryName']
).size().reset_index(name='total_film_acteurs').sort_values(by="total_film_acteurs")

grouped_df
top_acteurs_decennie = grouped_df.groupby(
    'cuts'
).apply(lambda x: x.nlargest(1, 'total_film_acteurs')).reset_index(drop=True)

decennies = top_acteurs_decennie['cuts']
noms_acteurs = top_acteurs_decennie['primaryName']
nombre_films = top_acteurs_decennie['total_film_acteurs']

plt.figure(figsize=(10, 6))
bars = plt.barh(
    decennies,
    nombre_films,
    color='#0047ab',
    edgecolor="black"
)
for bar, nom in zip(bars, noms_acteurs):
    plt.text(
        # bar.get_width() - 0.5, bar.get_y() + bar.get_height()/2 - 0.1,
        bar.get_width() - 0.5,
        bar.get_y() + bar.get_height()/2 - 0.1,
        # 1,
        # bar,
        f'{nom}',
        va='center',
        ha='right',
        color='white'
    )

plt.title('Acteur N°1 par Décennie et Nombre de Films Joués')
plt.xlabel('Nombre de Films Joués')
plt.ylabel('Décennie')
plt.show()


In [ ]:
actors_by_genre = df.explode('titre_genres').groupby(['primaryName', 'titre_genres']).size().reset_index(name='count')
top_actors_by_genre = actors_by_genre.sort_values('count', ascending=False).drop_duplicates('titre_genres')[:10][::-1]

plt.figure(figsize=(20, 10))
plt.barh(
    top_actors_by_genre['titre_genres'],
    top_actors_by_genre['count'],
    color='#006b3c',
    edgecolor = "black",
)
plt.xlabel('Nombre de Films')
plt.ylabel('Genres')
plt.title('Acteurs les plus fréquemment associés aux top 10 des genres')

for i, (count, name) in enumerate(zip(
    top_actors_by_genre['count'],
    top_actors_by_genre['primaryName'])):
    plt.text(1, i, name, color='white', va='center', ha='left', fontweight='bold')


plt.show()


In [ ]:
actors_film_count = df.groupby('primaryName').size().reset_index(name='film_count')

top_actors_film_count = actors_film_count.sort_values('film_count', ascending=False).head(10)[::-1]

plt.figure(figsize=(20, 10))
plt.barh(
    top_actors_film_count['primaryName'],
    top_actors_film_count['film_count'],
    color='#daa520',
    edgecolor="black",
)
plt.xlabel('Nombre de Films')
plt.ylabel('Acteurs')
plt.title('Acteurs ayant joué dans le plus grand nombre de films')
plt.show()


In [ ]:
actors_by_votes = df.groupby('primaryName')['rating_votes'].sum().reset_index()

top_actors_by_votes = actors_by_votes.sort_values('rating_votes', ascending=False).head(10)[::-1]

plt.figure(figsize=(20, 10))
plt.barh(
    top_actors_by_votes['primaryName'],
    top_actors_by_votes['rating_votes'],
    color='#008080',
    edgecolor =
    "black"
)
plt.xlabel('Total des votes')
plt.ylabel('Acteurs')
plt.title('Top 10 des acteurs dans des films ayant eu le plus de votes')
plt.ticklabel_format(style='plain', axis='x')
plt.show()


In [ ]:
# Je dégage pour les films d'animation les concerts et les docs.
genres_ = ['Animation', 'Music', 'Musical', 'Documentary']
df_filtre = df[df['titre_genres'].apply(lambda x: all(g not in x for g in genres_))]
# Je veux uniquement les films ayant eu le plus de vote, je filtre par quantile 0.75 sup only.
df_filtre = df_filtre[df_filtre['rating_votes'] > df_filtre['rating_votes'].quantile(0.75)]

# Groupe les films par la moyenne ratings, ne prend que le top 10
grouped_films = (
    df_filtre.groupby('titre_str')['rating_avg']
    .mean()
    .reset_index()
    .sort_values("rating_avg", ascending=False)
    .head(10)[::-1]
)

plt.figure(figsize=(20, 10))
plt.barh(
    grouped_films['titre_str'],
    grouped_films['rating_avg'],
    color='#e49b0f',
    edgecolor = "black"
)
plt.xlabel('Note Moyenne')
plt.ylabel('Films')
plt.title('Top 10 des films only')
plt.show()


In [ ]:
# Créer un dictionnaire contenant tout les noms des films avec leurs id
titre_dict = df[['titre_id', 'titre_str']].drop_duplicates().set_index('titre_id').to_dict()['titre_str']
# for k, v in titre_dict.items():
#     print(k, v)

In [ ]:
df1 = df.copy()

In [ ]:
import ast

# Fonction pour remplacer les identifiants dans une liste par les noms de titre
def replace_ids_with_titles(id_list):
    if isinstance(id_list, str):
        id_list = ast.literal_eval(id_list)
    return [titre_dict.get(titre_id, titre_id) for titre_id in id_list]

# Omn créer une nouvelle colonne pour le fun !
df1['person_film'] = df1['knownForTitles'].apply(replace_ids_with_titles)



In [ ]:
gerard = df1[df1["primaryName"].str.contains("Morgan")]
# gerard = gerard.copy()
gerard.head()

# on a une problemen, si les films ne sont pas dans la liste initial tconst, alors nous n'avons pas de modification, que faire ?
# remplacer par unknow ? supprimer les tt dans la liste et ne conserver que les films convertie ?

def if_tt_remove(id_list):
    return [t for t in id_list if not t.startswith("tt")]

In [ ]:
gerard.loc[:, 'person_film'] = gerard['person_film'].apply(if_tt_remove)
gerard